In [145]:
import duckdb
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [146]:
df = pd.read_csv("sample_users_with_first_posts.csv")
df

,did,handle,bio,created_at,first_post_created_at,first_post_body,is_reply,is_quote
0,did:plc:ooehugjick2vkzwlr64lephm,haleaziz.bsky.social,I cover DHS and immigration policy for the New...,2024-11-18 15:39:52.581,2024-11-16 20:04:59.812-05,If Trump-Miller-Homan want to shut down the ri...,False,False
1,did:plc:qy3kg6jvtlhenv5ojqurynwh,joshuacroke.bsky.social,Founder of 🏳️‍🌈 queerforcities.com / 🌈loveyour...,2024-11-15 11:45:12.223,2024-11-15 07:36:53.045-05,A they/them here for good trouble and out to m...,False,False
2,did:plc:ibqwicj6ersmvw36yldikozx,inspiringtimmy.bsky.social,Best known for watching the Queen's Gambit fro...,2024-11-06 08:11:15.881,2024-11-14 10:18:29.817-05,Foundations day 3\n\nlive now!,False,False
3,did:plc:f4ultoamz3y2m4drcjvhoecu,wilmonstouches.bsky.social,"same name but not rosh‘s ex | #OMAR : ”oh, tha...",2024-11-15 17:01:25.026,2024-11-15 17:20:49.418-05,it‘s cuz the nice ones switched to here and th...,True,False
4,did:plc:yzhdirwmonvaezndwnssoosd,kerrieneet.bsky.social,Wild places in the SW and beyond. Where to go?...,2024-11-20 01:32:25.972,2024-11-21 08:01:57.791-05,I would love to be added,True,False
...,...,...,...,...,...,...,...,...
23330,did:plc:67yljp3x2ggfmtouqhzm23ax,quigonejinn.bsky.social,NaN,2023-07-25 19:58:17.493,2021-08-13 10:03:56-04,"and yeah, I believe, deeply, that’s the kind o...",True,False
23331,did:plc:64vgrlkpc25gq7dzgv4wk4fz,raywillmott.bsky.social,Some Welsh Brand Manager in games @rawfury.bsk...,2023-07-23 23:19:36.113,2023-07-26 18:24:00.221-04,Made a curry tonight. One mouthful later and I...,False,False
23332,did:plc:ld6amztvyqa6dkyfpuw7agz3,madrigal.bsky.social,"I have a shiny, smooth coat and a precise, int...",2023-07-04 16:56:13.032,2023-05-04 10:50:34.133-04,Being an earthworm would be awesome. If you ge...,False,False
23333,did:plc:ifcposy3e6frphvetobi7mva,foxpopvli.bsky.social,"🔸 Anthros, Cars and Nostalgia\n🔸 100% Natural ...",2023-07-19 23:27:01.452,2023-07-04 08:53:40.821-04,The fox waifu <3\n\n#art #furry,False,False


In [147]:
new_features_df = df[['did', 'created_at', 'is_reply', 'is_quote']]
new_features_df

,did,created_at,is_reply,is_quote
0,did:plc:ooehugjick2vkzwlr64lephm,2024-11-18 15:39:52.581,False,False
1,did:plc:qy3kg6jvtlhenv5ojqurynwh,2024-11-15 11:45:12.223,False,False
2,did:plc:ibqwicj6ersmvw36yldikozx,2024-11-06 08:11:15.881,False,False
3,did:plc:f4ultoamz3y2m4drcjvhoecu,2024-11-15 17:01:25.026,True,False
4,did:plc:yzhdirwmonvaezndwnssoosd,2024-11-20 01:32:25.972,True,False
...,...,...,...,...
23330,did:plc:67yljp3x2ggfmtouqhzm23ax,2023-07-25 19:58:17.493,True,False
23331,did:plc:64vgrlkpc25gq7dzgv4wk4fz,2023-07-23 23:19:36.113,False,False
23332,did:plc:ld6amztvyqa6dkyfpuw7agz3,2023-07-04 16:56:13.032,False,False
23333,did:plc:ifcposy3e6frphvetobi7mva,2023-07-19 23:27:01.452,False,False


In [148]:
jobs = pd.read_csv('occupations.csv')
jobs

,Occupations
0,Accountant
1,Accounts Assistant
2,Accounts Clerk
3,Accounts Manager
4,Accounts Staff
...,...
1150,Youth Hostel Warden
1151,Youth Worker
1152,Zoo Keeper
1153,Zoo Manager


In [149]:
counts = np.zeros(jobs['Occupations'].shape[0])
job_index = 0
for job in jobs['Occupations']:
    count = 0
    for i in range(df['bio'].shape[0]):
        bio = df['bio'][i]
        if type(bio) is str and job in bio:
            count += 1
    print(count, job)
    counts[job_index] = count
    job_index += 1
jobs['count'] = counts

1 Accountant
0 Accounts Assistant
0 Accounts Clerk
0 Accounts Manager
0 Accounts Staff
0 Acoustic Engineer
101 Actor
17 Actress
1 Actuary
0 Acupuncturist
0 Adjustor
0 Administration Assistant
0 Administration Clerk
0 Administration Manager
0 Administration Staff
4 Administrator
0 Advertising Agent
0 Advertising Assistant
0 Advertising Clerk
0 Advertising Contractor
0 Advertising Executive
0 Advertising Manager
0 Advertising Staff
0 Aerial Erector
0 Aerobic Instructor
0 Aeronautical Engineer
59 Agent
0 Air Traffic Controller
0 Aircraft Designer
0 Aircraft Engineer
0 Aircraft Maintenance Engineer
0 Aircraft Surface Finisher
1 Airman
0 Airport Controller
0 Airport Manager
0 Almoner
0 Ambulance Controller
0 Ambulance Crew
0 Ambulance Driver
0 Amusement Arcade Worker
1 Anaesthetist
66 Analyst
0 Analytical Chemist
0 Animal Breeder
15 Anthropologist
0 Antique Dealer
0 Applications Engineer
0 Applications Programmer
0 Arbitrator
0 Arborist
22 Archaeologist
25 Architect
9 Archivist
1 Area Manag

In [150]:
jobs = jobs.loc[jobs['count'] > 0]

cutoff = 5
jobs['is_common'] = jobs['count'] > cutoff

jobs['feature_name'] = np.vectorize(str.replace)('is_'+jobs['Occupations'], ' ', '_')

jobs

/var/folders/1f/m2_9y_b17tsfg0cq6sk57v000000gn/T/ipykernel_62240/2891583624.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['is_common'] = jobs['count'] > cutoff
/var/folders/1f/m2_9y_b17tsfg0cq6sk57v000000gn/T/ipykernel_62240/2891583624.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['feature_name'] = np.vectorize(str.replace)('is_'+jobs['Occupations'], ' ', '_')


,Occupations,count,is_common,feature_name
0,Accountant,1.0,False,is_Accountant
6,Actor,101.0,True,is_Actor
7,Actress,17.0,True,is_Actress
8,Actuary,1.0,False,is_Actuary
15,Administrator,4.0,False,is_Administrator
...,...,...,...,...
1124,Warden,1.0,False,is_Warden
1129,Weaver,2.0,False,is_Weaver
1132,Welder,1.0,False,is_Welder
1147,Writer,582.0,True,is_Writer


In [151]:
job_feature_names = np.append(jobs.loc[jobs['is_common']]['feature_name'], ['is_Other', 'is_None'])
new_features_df[job_feature_names] = 0
new_features_df

/var/folders/1f/m2_9y_b17tsfg0cq6sk57v000000gn/T/ipykernel_62240/2669877003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_features_df[job_feature_names] = 0
/var/folders/1f/m2_9y_b17tsfg0cq6sk57v000000gn/T/ipykernel_62240/2669877003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_features_df[job_feature_names] = 0
/var/folders/1f/m2_9y_b17tsfg0cq6sk57v000000gn/T/ipykernel_62240/2669877003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,did,created_at,is_reply,is_quote,is_Actor,is_Actress,is_Agent,is_Analyst,is_Anthropologist,is_Archaeologist,...,is_Steward,is_Student,is_Teacher,is_Therapist,is_Translator,is_Treasurer,is_Tutor,is_Writer,is_Other,is_None
0,did:plc:ooehugjick2vkzwlr64lephm,2024-11-18 15:39:52.581,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,did:plc:qy3kg6jvtlhenv5ojqurynwh,2024-11-15 11:45:12.223,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,did:plc:ibqwicj6ersmvw36yldikozx,2024-11-06 08:11:15.881,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,did:plc:f4ultoamz3y2m4drcjvhoecu,2024-11-15 17:01:25.026,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,did:plc:yzhdirwmonvaezndwnssoosd,2024-11-20 01:32:25.972,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23330,did:plc:67yljp3x2ggfmtouqhzm23ax,2023-07-25 19:58:17.493,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23331,did:plc:64vgrlkpc25gq7dzgv4wk4fz,2023-07-23 23:19:36.113,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23332,did:plc:ld6amztvyqa6dkyfpuw7agz3,2023-07-04 16:56:13.032,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23333,did:plc:ifcposy3e6frphvetobi7mva,2023-07-19 23:27:01.452,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
for user_index, user_row in df.iterrows():
    bio = user_row['bio']
    found_any_job = False
    found_rare_job = False
    if type(bio) is str:
        for job_index, job_row in jobs.iterrows():
            if job_row['Occupations'] in bio:
                found_any_job = True
                if job_row['is_common']:
                    new_features_df.loc[user_index, job_row['feature_name']] = 1
                else:
                    new_features_df.loc[user_index, 'is_Other'] = 1
    if not found_any_job:
        new_features_df.loc[user_index, 'is_None'] = 1
new_features_df

,did,created_at,is_reply,is_quote,is_Actor,is_Actress,is_Agent,is_Analyst,is_Anthropologist,is_Archaeologist,...,is_Steward,is_Student,is_Teacher,is_Therapist,is_Translator,is_Treasurer,is_Tutor,is_Writer,is_Other,is_None
0,did:plc:ooehugjick2vkzwlr64lephm,2024-11-18 15:39:52.581,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,did:plc:qy3kg6jvtlhenv5ojqurynwh,2024-11-15 11:45:12.223,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,did:plc:ibqwicj6ersmvw36yldikozx,2024-11-06 08:11:15.881,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,did:plc:f4ultoamz3y2m4drcjvhoecu,2024-11-15 17:01:25.026,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,did:plc:yzhdirwmonvaezndwnssoosd,2024-11-20 01:32:25.972,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23330,did:plc:67yljp3x2ggfmtouqhzm23ax,2023-07-25 19:58:17.493,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23331,did:plc:64vgrlkpc25gq7dzgv4wk4fz,2023-07-23 23:19:36.113,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
23332,did:plc:ld6amztvyqa6dkyfpuw7agz3,2023-07-04 16:56:13.032,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23333,did:plc:ifcposy3e6frphvetobi7mva,2023-07-19 23:27:01.452,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [153]:
open_reg_date = datetime.datetime.fromisocalendar(2024, 6, 2)
new_features_df['before_open'] = (np.vectorize(datetime.datetime.fromisoformat)(df['created_at']) - open_reg_date) < datetime.timedelta(0)

In [154]:
new_features_df['default_domain'] = np.vectorize(str.endswith)(df['handle'], '.bsky.social')

In [155]:
new_features_df = new_features_df.rename(columns={"is_reply": "first_is_reply", "is_quote": "first_is_quote"})
new_features_df

,did,created_at,first_is_reply,first_is_quote,is_Actor,is_Actress,is_Agent,is_Analyst,is_Anthropologist,is_Archaeologist,...,is_Teacher,is_Therapist,is_Translator,is_Treasurer,is_Tutor,is_Writer,is_Other,is_None,before_open,default_domain
0,did:plc:ooehugjick2vkzwlr64lephm,2024-11-18 15:39:52.581,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,False,True
1,did:plc:qy3kg6jvtlhenv5ojqurynwh,2024-11-15 11:45:12.223,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,False,True
2,did:plc:ibqwicj6ersmvw36yldikozx,2024-11-06 08:11:15.881,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,False,True
3,did:plc:f4ultoamz3y2m4drcjvhoecu,2024-11-15 17:01:25.026,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,False,True
4,did:plc:yzhdirwmonvaezndwnssoosd,2024-11-20 01:32:25.972,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23330,did:plc:67yljp3x2ggfmtouqhzm23ax,2023-07-25 19:58:17.493,True,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,True,True
23331,did:plc:64vgrlkpc25gq7dzgv4wk4fz,2023-07-23 23:19:36.113,False,False,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,True,True
23332,did:plc:ld6amztvyqa6dkyfpuw7agz3,2023-07-04 16:56:13.032,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,True,True
23333,did:plc:ifcposy3e6frphvetobi7mva,2023-07-19 23:27:01.452,False,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,True,True


In [157]:
new_features_df.to_csv('nb_features.csv', index=False)